In [1]:
%load_ext autoreload
%autoreload 2

%matplotlib inline

In [2]:
import numpy as np
import pandas as pd
from pathlib import Path

In [3]:
path=Path('/kaggle/data_science_bowl')
path

PosixPath('/kaggle/data_science_bowl')

### Read Data

In [4]:
def read_data():
    train_df = pd.read_csv(path/'train.csv')
    test_df = pd.read_csv(path/'test.csv')
    train_labels_df = pd.read_csv(path/'train_labels.csv')
    sample_submission_df = pd.read_csv(path/'sample_submission.csv')
    return train_df, test_df, train_labels_df, sample_submission_df

In [5]:
train_df, test_df, train_labels_df, sample_submission_df = read_data()

In [6]:
train_df.head()

,event_id,game_session,timestamp,event_data,installation_id,event_count,event_code,game_time,title,type,world
0,27253bdc,45bb1e1b6b50c07b,2019-09-06T17:53:46.937Z,"{""event_code"": 2000, ""event_count"": 1}",0001e90f,1,2000,0,Welcome to Lost Lagoon!,Clip,NONE
1,27253bdc,17eeb7f223665f53,2019-09-06T17:54:17.519Z,"{""event_code"": 2000, ""event_count"": 1}",0001e90f,1,2000,0,Magma Peak - Level 1,Clip,MAGMAPEAK
2,77261ab5,0848ef14a8dc6892,2019-09-06T17:54:56.302Z,"{""version"":""1.0"",""event_count"":1,""game_time"":0...",0001e90f,1,2000,0,Sandcastle Builder (Activity),Activity,MAGMAPEAK
3,b2dba42b,0848ef14a8dc6892,2019-09-06T17:54:56.387Z,"{""description"":""Let's build a sandcastle! Firs...",0001e90f,2,3010,53,Sandcastle Builder (Activity),Activity,MAGMAPEAK
4,1bb5fbdb,0848ef14a8dc6892,2019-09-06T17:55:03.253Z,"{""description"":""Let's build a sandcastle! Firs...",0001e90f,3,3110,6972,Sandcastle Builder (Activity),Activity,MAGMAPEAK


In [7]:
test_df.head()

,event_id,game_session,timestamp,event_data,installation_id,event_count,event_code,game_time,title,type,world
0,27253bdc,0ea9ecc81a565215,2019-09-10T16:50:24.910Z,"{""event_code"": 2000, ""event_count"": 1}",00abaee7,1,2000,0,Welcome to Lost Lagoon!,Clip,NONE
1,27253bdc,c1ea43d8b8261d27,2019-09-10T16:50:55.503Z,"{""event_code"": 2000, ""event_count"": 1}",00abaee7,1,2000,0,Magma Peak - Level 1,Clip,MAGMAPEAK
2,27253bdc,7ed86c6b72e725e2,2019-09-10T16:51:51.805Z,"{""event_code"": 2000, ""event_count"": 1}",00abaee7,1,2000,0,Magma Peak - Level 2,Clip,MAGMAPEAK
3,27253bdc,7e516ace50e7fe67,2019-09-10T16:53:12.825Z,"{""event_code"": 2000, ""event_count"": 1}",00abaee7,1,2000,0,Crystal Caves - Level 1,Clip,CRYSTALCAVES
4,7d093bf9,a022c3f60ba547e7,2019-09-10T16:54:12.115Z,"{""version"":""1.0"",""round"":0,""event_count"":1,""ga...",00abaee7,1,2000,0,Chow Time,Game,CRYSTALCAVES


In [8]:
train_labels_df.head()

,game_session,installation_id,title,num_correct,num_incorrect,accuracy,accuracy_group
0,6bdf9623adc94d89,0006a69f,Mushroom Sorter (Assessment),1,0,1.0,3
1,77b8ee947eb84b4e,0006a69f,Bird Measurer (Assessment),0,11,0.0,0
2,901acc108f55a5a1,0006a69f,Mushroom Sorter (Assessment),1,0,1.0,3
3,9501794defd84e4d,0006a69f,Mushroom Sorter (Assessment),1,1,0.5,2
4,a9ef3ecb3d1acc6a,0006a69f,Bird Measurer (Assessment),1,0,1.0,3


In [9]:
installation_id_cnt = train_labels_df.groupby('installation_id')['installation_id'].size()
installation_id_cnt[installation_id_cnt > 1]

installation_id
0006a69f    5
0006c192    3
001d0ed0    5
002db7e3    9
003372b0    6
           ..
ff90db99    7
ff9305d7    4
ff9715db    9
ffc90c32    6
ffeb0b1b    3
Name: installation_id, Length: 2587, dtype: int64

In [10]:
train_labels_df_acc_group = train_labels_df.groupby(['installation_id']).agg({'accuracy_group': ['mean', 'std', list]})
train_labels_df_acc_group.columns = ['mean', 'std', 'list']
train_labels_df_acc_group['mean_rounded'] = train_labels_df_acc_group['mean'].apply(lambda x: round(x))
train_labels_df_acc_group.head(20)

,mean,std,list,mean_rounded
installation_id,,,,
0006a69f,2.200000,1.303840,"[3, 0, 3, 2, 3]",2
0006c192,1.666667,1.527525,"[3, 2, 0]",2
00129856,3.000000,NaN,[3],3
001d0ed0,1.600000,1.516575,"[0, 0, 3, 2, 3]",2
00225f67,0.000000,NaN,[0],0
00279ac5,0.000000,NaN,[0],0
002db7e3,1.666667,1.224745,"[2, 3, 3, 3, 1, 0, 2, 1, 0]",2
003372b0,2.333333,1.211060,"[3, 3, 2, 3, 0, 3]",2
004c2091,1.500000,1.732051,"[0, 3, 0, 3]",2


In [11]:
sample_submission_df.head()

,installation_id,accuracy_group
0,00abaee7,3
1,01242218,3
2,017c5718,3
3,01a44906,3
4,01bc6cb6,3


In [12]:
sample_submission_df.groupby(['installation_id'])[['installation_id']].count()

,installation_id
installation_id,
00abaee7,1
01242218,1
017c5718,1
01a44906,1
01bc6cb6,1
...,...
fee254cf,1
ff57e602,1
ffc73fb2,1


In [13]:
sample_session = '6bdf9623adc94d89'

In [14]:
train_labels_df[train_labels_df['game_session'] == sample_session]

,game_session,installation_id,title,num_correct,num_incorrect,accuracy,accuracy_group
0,6bdf9623adc94d89,0006a69f,Mushroom Sorter (Assessment),1,0,1.0,3


In [15]:
train_df[train_df['game_session'] == sample_session]

,event_id,game_session,timestamp,event_data,installation_id,event_count,event_code,game_time,title,type,world
2768,3bfd1a65,6bdf9623adc94d89,2019-08-06T05:37:50.020Z,"{""version"":""1.0"",""event_count"":1,""game_time"":0...",0006a69f,1,2000,0,Mushroom Sorter (Assessment),Assessment,TREETOPCITY
2769,db02c830,6bdf9623adc94d89,2019-08-06T05:37:50.078Z,"{""event_count"":2,""game_time"":77,""event_code"":2...",0006a69f,2,2025,77,Mushroom Sorter (Assessment),Assessment,TREETOPCITY
2770,a1e4395d,6bdf9623adc94d89,2019-08-06T05:37:50.082Z,"{""description"":""Pull three mushrooms out of th...",0006a69f,3,3010,77,Mushroom Sorter (Assessment),Assessment,TREETOPCITY
2771,7da34a02,6bdf9623adc94d89,2019-08-06T05:37:52.799Z,"{""coordinates"":{""x"":199,""y"":484,""stage_width"":...",0006a69f,4,4070,2784,Mushroom Sorter (Assessment),Assessment,TREETOPCITY
2772,28ed704e,6bdf9623adc94d89,2019-08-06T05:37:53.631Z,"{""height"":1,""coordinates"":{""x"":171,""y"":519,""st...",0006a69f,5,4025,3625,Mushroom Sorter (Assessment),Assessment,TREETOPCITY
2773,a52b92d5,6bdf9623adc94d89,2019-08-06T05:37:53.632Z,"{""description"":""Pull three mushrooms out of th...",0006a69f,6,3110,3625,Mushroom Sorter (Assessment),Assessment,TREETOPCITY
2774,9d29771f,6bdf9623adc94d89,2019-08-06T05:37:53.635Z,"{""description"":""That's one!"",""identifier"":""Dot...",0006a69f,7,3021,3625,Mushroom Sorter (Assessment),Assessment,TREETOPCITY
2775,c74f40cd,6bdf9623adc94d89,2019-08-06T05:37:54.253Z,"{""description"":""That's one!"",""identifier"":""Dot...",0006a69f,8,3121,4250,Mushroom Sorter (Assessment),Assessment,TREETOPCITY
2776,28ed704e,6bdf9623adc94d89,2019-08-06T05:37:54.930Z,"{""height"":2,""coordinates"":{""x"":496,""y"":502,""st...",0006a69f,9,4025,4925,Mushroom Sorter (Assessment),Assessment,TREETOPCITY
2777,9d29771f,6bdf9623adc94d89,2019-08-06T05:37:54.934Z,"{""description"":""two..."",""identifier"":""Dot_Two""...",0006a69f,10,3021,4925,Mushroom Sorter (Assessment),Assessment,TREETOPCITY


In [16]:
# df_event_count = train_df.groupby(['installation_id'])['event_id'].count().reset_index()
df_event_count = train_df.groupby(['installation_id', 'event_id']).agg({'event_id': ['count']})
df_event_count.columns = ['count']

In [17]:
df_event_count.head(8)

count
installation_id event_id       
0001e90f        08fd73f3     57
                1325467d    117
                16dffff1      3
                1996c610     13
                1bb5fbdb     62
                26fd2d99     57
                27253bdc      5
                2b9272f4     15

In [18]:
df_event_count_pivot = df_event_count.pivot_table(index = 'installation_id', columns = ['event_id'], values = 'count')
df_event_count_pivot = df_event_count_pivot.fillna(0)

In [19]:
df_event_count_pivot

event_id,003cd2ee,0086365d,00c73085,01ca3a3c,022b4259,02a42007,0330ab6a,0413e89d,04df9b66,05ad839b,...,f56e0afc,f5b8c21a,f6947f54,f71c4741,f7e47413,f806dc10,f93fc684,fbaf3456,fcfdffb6,fd20ea40
installation_id,,,,,,,,,,,,,,,,,,,,,
0001e90f,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,66.0,66.0,4.0,0.0,0.0,0.0,0.0
000447c4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0
0006a69f,0.0,0.0,0.0,0.0,24.0,70.0,0.0,29.0,0.0,0.0,...,2.0,3.0,2.0,48.0,45.0,9.0,0.0,18.0,45.0,0.0
0006c192,0.0,1.0,3.0,0.0,8.0,0.0,9.0,10.0,0.0,0.0,...,1.0,0.0,1.0,15.0,14.0,0.0,1.0,14.0,35.0,0.0
0009a5a9,0.0,0.0,9.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
fff64664,0.0,0.0,0.0,0.0,0.0,0.0,57.0,0.0,0.0,0.0,...,0.0,0.0,0.0,17.0,17.0,0.0,2.0,0.0,0.0,0.0
fff70c1e,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,10.0,10.0,0.0,0.0,0.0,0.0,0.0
fff744ce,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


## Feature Engineering

In [20]:
def extract_time_features(df):
    df['timestamp'] = pd.to_datetime(df['timestamp'])
    df['date'] = df['timestamp'].dt.date
    df['month'] = df['timestamp'].dt.month
    df['hour'] = df['timestamp'].dt.hour
    df['year'] = df['timestamp'].dt.year
    df['dayofweek'] = df['timestamp'].dt.dayofweek
    df['weekofyear'] = df['timestamp'].dt.weekofyear
    df['dayofyear'] = df['timestamp'].dt.dayofyear
    df['quarter'] = df['timestamp'].dt.quarter
    df['is_month_start'] = df['timestamp'].dt.is_month_start 
    return df
    
def get_numeric_columns(df, column):
    df = df.groupby('installation_id').agg({f'{column}': ['mean', 'sum', 'min', 'max', 'std', 'skew', 'median']})
    df[column].fillna(df[column].mean(), inplace = True)
    df.columns = [f'{column}_mean', f'{column}_sum', f'{column}_min', f'{column}_max', f'{column}_std', f'{column}_skew', f'{column}_median']
    return df

def get_object_columns(df, columns):
    df = df.groupby(['installation_id', columns])['event_id'].count().reset_index()
    df = df.pivot_table(index = 'installation_id', columns = [columns], values = 'event_id')
    df.columns = list(df.columns)
    df.fillna(0, inplace = True)
    return df

def get_numeric_columns_add(df, agg_column, column):
    df = df.groupby(['installation_id', agg_column]).agg({f'{column}': ['mean', 'sum', 'min', 'max', 'std', 'skew']}).reset_index()
    df = df.pivot_table(index = 'installation_id', columns = [agg_column], values = [col for col in df.columns if col not in ['installation_id', 'type']])
    df[column].fillna(df[column].mean(), inplace = True)
    df.columns = list(df.columns)
    return df

def get_event_counts(df):
    df_event_count = train_df.groupby(['installation_id', 'event_id']).agg({'event_id': ['count']})
    df_event_count.columns = ['count']
    df_event_count_pivot = df_event_count.pivot_table(index = 'installation_id', columns = ['event_id'], values = 'count')
    df_event_count_pivot = df_event_count_pivot.fillna(0)
    return df_event_count_pivot

def perform_features_engineering(train_df, test_df, train_labels_df):
    print('Perform features engineering')
    numerical_columns = ['game_time']
    categorical_columns = ['type', 'world']

    comp_train_df = pd.DataFrame({'installation_id': train_df['installation_id'].unique()})
    comp_train_df.set_index('installation_id', inplace = True)
    comp_test_df = pd.DataFrame({'installation_id': test_df['installation_id'].unique()})
    comp_test_df.set_index('installation_id', inplace = True)

    test_df = extract_time_features(test_df)
    train_df = extract_time_features(train_df)

    for i in numerical_columns:
        comp_train_df = comp_train_df.merge(get_numeric_columns(train_df, i), left_index = True, right_index = True)
        comp_test_df = comp_test_df.merge(get_numeric_columns(test_df, i), left_index = True, right_index = True)
        
    for i in categorical_columns:
        comp_train_df = comp_train_df.merge(get_object_columns(train_df, i), left_index = True, right_index = True)
        comp_test_df = comp_test_df.merge(get_object_columns(test_df, i), left_index = True, right_index = True)
        
    comp_train_df = comp_train_df.merge(get_event_counts(train_df), left_index = True, right_index = True)
        
#     for i in categorical_columns:
#         for j in numerical_columns:
#             comp_train_df = comp_train_df.merge(get_numeric_columns_add(train_df, i, j), left_index = True, right_index = True)
#             comp_test_df = comp_test_df.merge(get_numeric_columns_add(test_df, i, j), left_index = True, right_index = True)
        
    comp_train_df.reset_index(inplace = True)
    comp_test_df.reset_index(inplace = True)
    
#     # get the mode of the title
#     labels_map = dict(train_labels_df.groupby('title')['accuracy_group'].agg(lambda x:x.value_counts().index[0]))
#     # merge target
#     labels = train_labels_df[['installation_id', 'title', 'accuracy_group']]
#     # replace title with the mode
#     labels['title'] = labels['title'].map(labels_map)
#     # get title from the test set
#     comp_test_df['title'] = test_df.groupby('installation_id').last()['title'].map(labels_map).reset_index(drop = True)
#     # join train with labels
    train_labels_df_acc_group = train_labels_df.groupby(['installation_id']).agg({'accuracy_group': ['mean']})
    train_labels_df_acc_group.columns = ['accuracy_group']
    train_labels_df_acc_group['accuracy_group'] = train_labels_df_acc_group['accuracy_group'].apply(lambda x: round(x))
    
    comp_train_df = train_labels_df_acc_group.merge(comp_train_df, on = 'installation_id', how = 'left')
#     title_counter = {e[0]:i for i,e in enumerate (dict(comp_train_df['title'].value_counts()).items())}
#     title_map = {'title': title_counter}
#     comp_train_df.replace(title_map, inplace=True)
    
    return comp_train_df, comp_test_df

In [21]:
comp_train_df, comp_test_df = perform_features_engineering(train_df, test_df, train_labels_df)

Perform features engineering


/opt/anaconda3/envs/deeplearning/lib/python3.7/site-packages/pandas/core/generic.py:6287: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._update_inplace(new_data)


In [22]:
comp_train_x = comp_train_df[[i for i in comp_train_df.columns if i not in ['accuracy_group', 'installation_id', 'game_session']]]

In [23]:
comp_train_df

,installation_id,accuracy_group,game_time_mean,game_time_sum,game_time_min,game_time_max,game_time_std,game_time_skew,game_time_median,Activity,...,f56e0afc,f5b8c21a,f6947f54,f71c4741,f7e47413,f806dc10,f93fc684,fbaf3456,fcfdffb6,fd20ea40
0,0006a69f,2,82491.007366,313548319,0,1520600,159789.192183,7.504204,50155.0,1771.0,...,2.0,3.0,2.0,48.0,45.0,9.0,0.0,18.0,45.0,0.0
1,0006c192,2,90398.352968,201045937,0,395901,77779.338412,1.366377,69533.0,1206.0,...,1.0,0.0,1.0,15.0,14.0,0.0,1.0,14.0,35.0,0.0
2,00129856,3,99141.346988,82287318,0,370945,77906.465874,1.126816,76479.0,786.0,...,1.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,001d0ed0,2,77631.357004,79805035,0,393790,80681.341963,1.471436,47707.0,76.0,...,1.0,0.0,0.0,42.0,42.0,0.0,2.0,8.0,0.0,0.0
4,00225f67,0,70398.623824,67371483,0,192166,50706.362921,0.441866,62797.0,291.0,...,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,15.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3609,ff9305d7,0,106055.828434,418496299,0,445379,94619.434874,1.115646,71400.0,2486.0,...,2.0,0.0,2.0,0.0,0.0,4.0,2.0,13.0,0.0,0.0
3610,ff9715db,2,63466.335274,130804117,0,1589444,94560.021344,12.187013,48908.0,909.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,9.0,31.0,1.0
3611,ffc90c32,3,79080.932136,158478188,0,827160,109443.413015,4.028416,51465.0,479.0,...,1.0,3.0,2.0,50.0,50.0,3.0,2.0,10.0,10.0,1.0
3612,ffd2871d,3,173255.870612,424476883,0,1204349,193862.300262,2.677175,113377.5,1382.0,...,0.0,0.0,0.0,9.0,8.0,6.0,2.0,0.0,0.0,0.0


In [24]:
list(comp_train_df.columns)

['installation_id',
 'accuracy_group',
 'game_time_mean',
 'game_time_sum',
 'game_time_min',
 'game_time_max',
 'game_time_std',
 'game_time_skew',
 'game_time_median',
 'Activity',
 'Assessment',
 'Clip',
 'Game',
 'CRYSTALCAVES',
 'MAGMAPEAK',
 'NONE',
 'TREETOPCITY',
 '003cd2ee',
 '0086365d',
 '00c73085',
 '01ca3a3c',
 '022b4259',
 '02a42007',
 '0330ab6a',
 '0413e89d',
 '04df9b66',
 '05ad839b',
 '06372577',
 '070a5291',
 '08fd73f3',
 '08ff79ad',
 '0a08139c',
 '0ce40006',
 '0d18d96c',
 '0d1da71f',
 '0db6d71d',
 '119b5b02',
 '1325467d',
 '1340b8d7',
 '1375ccb7',
 '13f56524',
 '14de4c5d',
 '155f62a4',
 '1575e76c',
 '15a43e5b',
 '15ba1109',
 '15eb4a7d',
 '15f99afc',
 '160654fd',
 '16667cc5',
 '16dffff1',
 '17113b36',
 '17ca3959',
 '19967db1',
 '1996c610',
 '1af8be29',
 '1b54d27f',
 '1bb5fbdb',
 '1beb320a',
 '1c178d24',
 '1cc7cfca',
 '1cf54632',
 '1f19558b',
 '222660ff',
 '2230fab4',
 '250513af',
 '25fa8af4',
 '262136f4',
 '26a5a3dd',
 '26fd2d99',
 '27253bdc',
 '28520915',
 '28a4eb9a',


In [25]:
comp_train_x.head()

,game_time_mean,game_time_sum,game_time_min,game_time_max,game_time_std,game_time_skew,game_time_median,Activity,Assessment,Clip,...,f56e0afc,f5b8c21a,f6947f54,f71c4741,f7e47413,f806dc10,f93fc684,fbaf3456,fcfdffb6,fd20ea40
0,82491.007366,313548319,0,1520600,159789.192183,7.504204,50155.0,1771.0,261.0,37.0,...,2.0,3.0,2.0,48.0,45.0,9.0,0.0,18.0,45.0,0.0
1,90398.352968,201045937,0,395901,77779.338412,1.366377,69533.0,1206.0,343.0,32.0,...,1.0,0.0,1.0,15.0,14.0,0.0,1.0,14.0,35.0,0.0
2,99141.346988,82287318,0,370945,77906.465874,1.126816,76479.0,786.0,43.0,1.0,...,1.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,77631.357004,79805035,0,393790,80681.341963,1.471436,47707.0,76.0,202.0,38.0,...,1.0,0.0,0.0,42.0,42.0,0.0,2.0,8.0,0.0,0.0
4,70398.623824,67371483,0,192166,50706.362921,0.441866,62797.0,291.0,65.0,15.0,...,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,15.0,0.0


In [26]:
installation_id_cnt = comp_train_df.groupby('installation_id')['installation_id'].size()
installation_id_cnt[installation_id_cnt > 1]

Series([], Name: installation_id, dtype: int64)

In [27]:
comp_train_df[['installation_id', 'accuracy_group']]

,installation_id,accuracy_group
0,0006a69f,2
1,0006c192,2
2,00129856,3
3,001d0ed0,2
4,00225f67,0
...,...,...
3609,ff9305d7,0
3610,ff9715db,2
3611,ffc90c32,3
3612,ffd2871d,3


## Data Set and Data Loader

In [28]:
from torch.utils.data import DataLoader, Dataset, SequentialSampler, RandomSampler
import torch
from torch import nn

def convert_df_to_tensor(df):
    return torch.tensor(df.values)

class BowlDataSet(Dataset):
    def __init__(self, df_x: pd.DataFrame, df_y: pd.DataFrame):
        self.x = convert_df_to_tensor(df_x).float()
        self.y = convert_df_to_tensor(df_y).long()
        
    def __len__(self):
        return len(self.x)
    
    def __getitem__(self, i):
        return self.x[i], self.y[i]
    

In [29]:
m_count = comp_train_df.shape[0]
pct = 0.8
x_size = int(m_count * pct)
comp_train_y = comp_train_df['accuracy_group']
train_ds = BowlDataSet(comp_train_x[:x_size], comp_train_y[:x_size])
valid_ds = BowlDataSet(comp_train_x[x_size:], comp_train_y[x_size:])

In [30]:
len(train_ds), len(valid_ds), x_size

(2891, 723, 2891)

In [31]:
assert (len(train_ds) + len(valid_ds) == len(comp_train_df))

In [32]:
bs = 64

In [33]:
train_dl = DataLoader(train_ds, bs, shuffle=True, drop_last=True)
valid_dl = DataLoader(valid_ds, bs, shuffle=False)

In [34]:
batch_sample = next(iter(train_dl))

In [35]:
batch_sample[0].shape, batch_sample[1].shape

(torch.Size([64, 399]), torch.Size([64]))

## Model

In [36]:
m = comp_train_x.shape[1]
m

399

In [37]:
number_hidden = 600
number_hidden_2 = 300

In [38]:
categories = len(comp_train_df['accuracy_group'].unique())
categories

4

In [39]:
model = nn.Sequential(nn.Linear(m, number_hidden), 
                      nn.ReLU(inplace=True), 
                      nn.BatchNorm1d(number_hidden, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True),
                      nn.Linear(number_hidden, number_hidden_2), 
                      nn.ReLU(inplace=True), 
                      nn.BatchNorm1d(number_hidden_2, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True),
                      nn.Linear(number_hidden_2, categories))
model

Sequential(
  (0): Linear(in_features=399, out_features=600, bias=True)
  (1): ReLU(inplace=True)
  (2): BatchNorm1d(600, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (3): Linear(in_features=600, out_features=300, bias=True)
  (4): ReLU(inplace=True)
  (5): BatchNorm1d(300, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (6): Linear(in_features=300, out_features=4, bias=True)
)

## Loss and Optimizers

In [40]:
import torch.nn.functional as F
loss_func = F.cross_entropy
# loss_func = F.mse_loss

In [41]:
lr = 1e-2
lr

0.01

In [42]:
import torch.optim as optim
optimizer = optim.Adam(model.parameters(), lr, [0.9, 0.99])
optimizer

Adam (
Parameter Group 0
    amsgrad: False
    betas: [0.9, 0.99]
    eps: 1e-08
    lr: 0.01
    weight_decay: 0
)

## Accuracy functions

In [43]:
# quadratic weighted kappa
def qwk3(a1, a2, max_rat=3):
    '''
    a1 - ground truth
    a2 - predicted values
    '''
    assert(len(a1) == len(a2))
    a1 = np.asarray(a1, dtype=int)
    a2 = np.asarray(a2, dtype=int)
    hist1 = np.zeros((max_rat + 1, ))
    hist2 = np.zeros((max_rat + 1, ))
    o = 0
    for k in range(a1.shape[0]):
        i, j = a1[k], a2[k]
        hist1[i] += 1
        hist2[j] += 1
        o +=  (i - j) * (i - j)
    e = 0
    for i in range(max_rat + 1):
        for j in range(max_rat + 1):
            e += hist1[i] * hist2[j] * (i - j) * (i - j)
    e = e / a1.shape[0]
    return 1 - o / e

In [44]:
def mse(output, targ): 
    return (output.float() - targ.float()).pow(2).mean()

In [45]:
def accuracy(out, yb): 
    return (torch.argmax(out, dim=1)==yb).float().mean()

In [46]:
mse_text_output = torch.randn(32, 4).float()
targ = torch.randn(32).long()
mse_text_output_argmax = mse_text_output.argmax(-1)
mse(mse_text_output_argmax, targ)

tensor(5.1250)

In [47]:
qwk3(mse_text_output_argmax, targ)

-0.5619047619047619

In [48]:
accuracy(mse_text_output, targ)

tensor(0.3125)

In [49]:
"Normalize the continuous variables."
def normalize(df: pd.DataFrame, cont_names):
    "Compute the means and stds of `self.cont_names` columns to normalize them."
    means, stds = {},{}
    for n in cont_names:
        means[n], stds[n] = df[n].mean(), df[n].std()
        df[n] = (df[n]-means[n]) / (1e-7 + stds[n])

In [50]:
normalize(comp_train_df, ['game_time_mean', 'game_time_sum', 'game_time_min', 'game_time_max', 'game_time_std', 'game_time_skew', 'Activity', 'Assessment'])

In [51]:
comp_train_df

,installation_id,accuracy_group,game_time_mean,game_time_sum,game_time_min,game_time_max,game_time_std,game_time_skew,game_time_median,Activity,...,f56e0afc,f5b8c21a,f6947f54,f71c4741,f7e47413,f806dc10,f93fc684,fbaf3456,fcfdffb6,fd20ea40
0,0006a69f,2,-0.154965,-0.049452,0.0,-0.009008,-0.056330,1.495441,50155.0,0.833560,...,2.0,3.0,2.0,48.0,45.0,9.0,0.0,18.0,45.0,0.0
1,0006c192,2,-0.126062,-0.172084,0.0,-0.132177,-0.130038,-0.210843,69533.0,0.346549,...,1.0,0.0,1.0,15.0,14.0,0.0,1.0,14.0,35.0,0.0
2,00129856,3,-0.094105,-0.301535,0.0,-0.134910,-0.129924,-0.277440,76479.0,-0.015477,...,1.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,001d0ed0,2,-0.172728,-0.304240,0.0,-0.132408,-0.127430,-0.181637,47707.0,-0.627473,...,1.0,0.0,0.0,42.0,42.0,0.0,2.0,8.0,0.0,0.0
4,00225f67,0,-0.199164,-0.317793,0.0,-0.154488,-0.154370,-0.467852,62797.0,-0.442151,...,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,15.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3609,ff9305d7,0,-0.068832,0.064945,0.0,-0.126758,-0.114903,-0.280545,71400.0,1.449866,...,2.0,0.0,2.0,0.0,0.0,4.0,2.0,13.0,0.0,0.0
3610,ff9715db,2,-0.224503,-0.248650,0.0,-0.001469,-0.114956,2.797237,48908.0,0.090545,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,9.0,31.0,1.0
3611,ffc90c32,3,-0.167429,-0.218484,0.0,-0.084949,-0.101579,0.529190,51465.0,-0.280101,...,1.0,3.0,2.0,50.0,50.0,3.0,2.0,10.0,10.0,1.0
3612,ffd2871d,3,0.176794,0.071464,0.0,-0.043642,-0.025706,0.153552,113377.5,0.498255,...,0.0,0.0,0.0,9.0,8.0,6.0,2.0,0.0,0.0,0.0


## Training Loop

In [52]:
def fit(epochs: int, model: nn.Sequential, loss_func, opt, train_dl, valid_dl):
    for epoch in range(epochs):
        model.train()
        tot_train_loss = 0.
        for xb, yb in train_dl:
            loss = loss_func(model(xb), yb)
            tot_train_loss += loss
            loss.backward()
            opt.step()
            opt.zero_grad()
            
        model.eval()
        with torch.no_grad():
            tot_loss, tot_mse, tot_acc, tot_acc_qwk = 0., 0., 0., 0.
            for xb, yb in valid_dl:
                pred = model(xb)
                tot_loss += loss_func(pred, yb)
                tot_acc += accuracy(pred, yb)
                tot_acc_qwk += qwk3(pred.argmax(-1), yb)
            nv = len(valid_dl)
            print(f'{epoch} trainloss: {tot_train_loss / len(train_dl)} valid loss: {tot_loss / nv} qwk: {tot_acc_qwk / nv} acc: {tot_acc / nv}')

In [53]:
fit(1000, model, loss_func, optimizer, train_dl, valid_dl)

0 trainloss: 1.4740320444107056 valid loss: 1.3305002450942993 qwk: 0.11503737585120394 acc: 0.4639528691768646
1 trainloss: 1.3526002168655396 valid loss: 1.4450840950012207 qwk: 0.09289715449179743 acc: 0.4035772979259491
2 trainloss: 1.3140416145324707 valid loss: 1.2870593070983887 qwk: 0.040422898026264566 acc: 0.39007675647735596
3 trainloss: 1.3198291063308716 valid loss: 1.2654484510421753 qwk: 0.0709436283069153 acc: 0.39137884974479675
4 trainloss: 1.3441630601882935 valid loss: 1.325442910194397 qwk: 0.043545107725781323 acc: 0.3931606113910675
5 trainloss: 1.295938491821289 valid loss: 1.2847813367843628 qwk: 0.1265752111503974 acc: 0.4756716191768646
6 trainloss: 1.3103506565093994 valid loss: 1.2870935201644897 qwk: 0.07037006194418453 acc: 0.3762335479259491
7 trainloss: 1.3130918741226196 valid loss: 1.2964341640472412 qwk: 0.13746102265040946 acc: 0.3975466191768646
8 trainloss: 1.2964675426483154 valid loss: 1.287956714630127 qwk: 0.060924003938765264 acc: 0.384046047

73 trainloss: 1.2382910251617432 valid loss: 1.2582305669784546 qwk: 0.03307125259483945 acc: 0.39268091320991516
74 trainloss: 1.2384393215179443 valid loss: 1.2453147172927856 qwk: 0.11601349942818402 acc: 0.4478481113910675
75 trainloss: 1.2396517992019653 valid loss: 1.4030729532241821 qwk: 0.016656411503756668 acc: 0.39268091320991516
76 trainloss: 1.2449220418930054 valid loss: 4.3823113441467285 qwk: 0.0037392990808512716 acc: 0.39007675647735596
77 trainloss: 1.2409801483154297 valid loss: 1.3207148313522339 qwk: 0.015126913463325786 acc: 0.39137884974479675
78 trainloss: 1.2390213012695312 valid loss: 1.2908122539520264 qwk: 0.00753936482519349 acc: 0.39007675647735596
79 trainloss: 1.2399303913116455 valid loss: 1.2415977716445923 qwk: 0.033639326435894755 acc: 0.43174341320991516
80 trainloss: 1.2397736310958862 valid loss: 1.2743924856185913 qwk: 0.011462906072678395 acc: 0.39007675647735596
81 trainloss: 1.2444514036178589 valid loss: 1.9852185249328613 qwk: -0.00384824955

145 trainloss: 1.2507128715515137 valid loss: 1.2712699174880981 qwk: 0.017670224230520514 acc: 0.39007675647735596
146 trainloss: 1.2526298761367798 valid loss: 1.246366262435913 qwk: 0.10654666899318273 acc: 0.47827577590942383
147 trainloss: 1.250167727470398 valid loss: 1.2726035118103027 qwk: 0.02391392537905875 acc: 0.39137884974479675
148 trainloss: 1.251062273979187 valid loss: 1.2706483602523804 qwk: 0.0103697416264772 acc: 0.38617050647735596
149 trainloss: 1.2548788785934448 valid loss: 4.080495834350586 qwk: 0.10226229300565226 acc: 0.4660772979259491
150 trainloss: 1.2516100406646729 valid loss: 2.330217123031616 qwk: 0.05396556337790299 acc: 0.3853481113910675
151 trainloss: 1.256532073020935 valid loss: 2.3782386779785156 qwk: 0.005261111768502987 acc: 0.38617050647735596
152 trainloss: 1.2510344982147217 valid loss: 1.9470891952514648 qwk: 0.003042806235061063 acc: 0.38617050647735596
153 trainloss: 1.2535629272460938 valid loss: 7.405074596405029 qwk: 0.005128464251915

KeyboardInterrupt: 